Data Ingestion

In [1]:
import os

In [2]:
%pwd

'c:\\Users\\EliteBook\\OneDrive\\Documents\\Chicken Disease Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\EliteBook\\OneDrive\\Documents\\Chicken Disease Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# THis is not the original data class, this decorator is being created for bulding my own custom return types.
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from CnnClassifier.constants import *
from CnnClassifier.utils.mutuals import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion
          create_directories([config.root_dir])
          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )

          return data_ingestion_config

In [8]:
import os
import boto3
# import urllib.request as request
import zipfile
from dotenv import load_dotenv
from CnnClassifier import logger
from CnnClassifier.utils.mutuals import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        load_dotenv()

        self.s3_client = boto3.client(
            "s3",
            aws_access_key_id=os.getenv("AWS_ACCESS_KEY"),
            aws_secret_access_key=os.getenv("AWS_SECRET_KEY"),
            region_name=os.getenv("REGION_NAME")
        )

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # Parse S3 URI
            s3_uri = self.config.source_URL  
            assert s3_uri.startswith("s3://"), "Invalid S3 URI"
            s3_uri_parts = s3_uri.replace("s3://", "").split("/", 1)
            bucket_name = s3_uri_parts[0]
            object_key = s3_uri_parts[1]

            self.s3_client.download_file(
                Bucket=bucket_name,
                Key=object_key,
                Filename=self.config.local_data_file
            )
            logger.info(f"{self.config.local_data_file} downloaded from S3!")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

Executing through Pipeline

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

{2025-05-22 22:05:47,138}: INFO: mutuals: yaml file: config\config.yaml loaded successfully}
{2025-05-22 22:05:47,138}: INFO: mutuals: yaml file: params.yaml loaded successfully}
{2025-05-22 22:05:47,145}: INFO: mutuals: created directory at: artifacts}
{2025-05-22 22:05:47,146}: INFO: mutuals: created directory at: artifacts/data_ingestion}
{2025-05-22 22:06:51,465}: INFO: 240432036: artifacts/data_ingestion/data.zip downloaded from S3!}
